In [1]:
#importing libraries
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#dataset
filename = 'C:\\Users\\HP\\Downloads\\data-mining\\datasetB.json'

In [3]:
#selecting successful theraphy
with open(filename, encoding="utf8") as f:
    data = json.load(f)
# trials under patients
trials_data = pd.json_normalize(data['Patients'], "trials", ["id", "name"], errors='ignore', record_prefix='_')
print(trials_data)

# conditions under patients
conditions_data = pd.json_normalize(data['Patients'], "conditions", ["id"], errors='ignore', record_prefix='_')
# print(conditions_data)

merge_trials_with_conditions = pd.merge(trials_data, conditions_data, how='left', left_on=['_condition', 'id'], right_on=['_id', 'id'])
# print(merge_trials_with_conditions)

#creating condition theraphy interaction matrix
therapy_matrix_CTI = merge_trials_with_conditions.pivot_table(index='_kind', columns='_therapy', values='_successful')
therapy_matrix_CTI.head()

       _condition      _end       _id    _start  _successful _therapy     id  \
0             pc3  20120109       tr1  20111219           86     Th49      0   
1             pc3  20120217       tr2  20120203           10     Th45      0   
2             pc3  20120404       tr3  20120330          100     Th45      0   
3             pc4  19650727       tr4  19650714          100     Th17      0   
4             pc5  19731019       tr5  19730919          100     Th47      0   
...           ...       ...       ...       ...          ...      ...    ...   
939962   pc540469  19770925  tr939963  19770925           49      Th7  99999   
939963   pc540469  19771029  tr939964  19770928          100     Th50  99999   
939964   pc540470  19770827  tr939965  19770823           20     Th28  99999   
939965   pc540472  19630803  tr939966  19630619           53     Th19  99999   
939966   pc540472  19631020  tr939967  19630917          100      Th8  99999   

                  name  
0        Thoma

_therapy,Th1,Th10,Th11,Th12,Th13,Th14,Th15,Th16,Th17,Th18,...,Th47,Th48,Th49,Th5,Th50,Th51,Th6,Th7,Th8,Th9
_kind,,,,,,,,,,,,,,,,,,,,,
Cond1,67.349206,64.268657,59.595745,55.404255,60.488889,66.732143,64.745098,63.833333,58.584906,61.818182,...,70.857143,67.176471,62.639344,66.566038,78.415094,67.811594,63.822581,71.550000,64.152778,61.226415
Cond10,61.938776,66.163636,69.414634,67.944444,65.796296,59.918367,74.272727,67.314815,70.590164,65.500000,...,79.584906,71.500000,69.636364,65.408163,69.719298,59.833333,66.150000,71.472727,69.808511,69.115385
Cond100,61.805970,68.581818,75.438596,64.614286,67.569231,65.382353,61.041667,66.129630,67.750000,63.118644,...,69.687500,60.946667,66.084746,72.375000,65.092593,69.070175,75.000000,67.573770,63.568627,66.836066
Cond101,70.363636,62.403846,72.086207,69.239437,73.963636,68.500000,70.803571,72.018868,78.000000,63.430556,...,61.285714,64.567568,69.854839,72.320000,68.524590,73.440000,71.966102,60.145455,69.592593,65.433962
Cond102,63.637681,63.888889,66.049180,75.907407,63.641509,70.930233,72.280000,69.400000,61.750000,67.038462,...,70.784615,54.941176,63.046875,65.283019,69.437500,65.786885,60.483871,65.787879,72.200000,67.915493


In [4]:
#findinf nearest neighbor 
n = 1
order = np.argsort(therapy_matrix_CTI.values, axis=1)[:, :n]
nearest_neighbor_df = therapy_matrix_CTI.apply(lambda x: pd.Series(x.sort_values(ascending=False)
                                      .iloc[:n].index,
                                      index=['top_{}'.format(i) for i in range(1, n + 1)]), axis=1)
nearest_neighbor_df.head()

,top_1
_kind,
Cond1,Th50
Cond10,Th42
Cond100,Th11
Cond101,Th17
Cond102,Th21


In [5]:
condition = 'Cond10'
#trials_data = trials_data[trials_data['_kind'] == condition]
trials_data = trials_data.groupby('_therapy')['_successful'].mean()

row_data = nearest_neighbor_df.loc[condition, :]
print('Top Recommended Therapies for ' + condition + ': ')
df_ranked = pd.DataFrame(columns=['Therapy', 'Success'])

for items in row_data:
    try:
        df_ranked = df_ranked.append({'Therapy': items, 'Success': trials_data[items]}, ignore_index=True)
    except:
        break
print(df_ranked['Therapy'][0])

Top Recommended Therapies for Cond10: 
Th42


In [6]:
#reading dataset
with open(filename, encoding="utf8") as f:
    data = json.load(f)

# trials under patients
trials_data = pd.json_normalize(data['Patients'], "trials", ["id", "name"], errors='ignore', record_prefix='_')
# print(trials_data)

# conditions under patients
conditions_data = pd.json_normalize(data['Patients'], "conditions", ["id"], errors='ignore', record_prefix='_')
# print(conditions_data)

merge_trials_with_conditions = pd.merge(trials_data, conditions_data, how='left', left_on=['_condition', 'id'], right_on=['_id', 'id'])
# print(merge_trials_with_conditions)
# output entity | _condition, _end, _id_x, _start, _cured, _diagnosed, _id_y, _isCured, _isTreated, name

columns = ['id', '_kind', '_therapy', '_successful']
#print(merge_trials_with_conditions[columns])
merged_data = merge_trials_with_conditions[columns]

#see the data description
merged_data.describe()


,_successful
count,939967.000000
mean,67.019765
std,33.760628
min,0.000000
25%,37.000000
50%,75.000000
75%,100.000000
max,100.000000


In [7]:
# creating mean reating of successful therapy
therapy_success_rate = pd.DataFrame(merged_data.groupby('_therapy')['_successful'].mean())
therapy_success_rate.head()

,_successful
_therapy,
Th1,67.074494
Th10,66.936326
Th11,67.170059
Th12,66.819908
Th13,66.973858


In [8]:
#creating number of therapy data
therapy_success_rate['number_of_successful_therapy'] = merged_data.groupby('_therapy')['_successful'].count()
therapy_success_rate.head()

,_successful,number_of_successful_therapy
_therapy,,
Th1,67.074494,18592
Th10,66.936326,18579
Th11,67.170059,18523
Th12,66.819908,18485
Th13,66.973858,18438


In [9]:
#Plotting the jointplot
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# sns.jointplot(x='_successful', y='number_of_successful_therapy', data=therapy_success_rate)

In [10]:
#creating patient therapy interaction matrix
therapy_matrix_PTI = merged_data.pivot_table(index='id', columns='_therapy', values='_successful')
therapy_matrix_PTI.head()

_therapy,Th1,Th10,Th11,Th12,Th13,Th14,Th15,Th16,Th17,Th18,...,Th47,Th48,Th49,Th5,Th50,Th51,Th6,Th7,Th8,Th9
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,...,100.0,NaN,86.0,NaN,58.0,NaN,NaN,NaN,NaN,NaN
2,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN
3,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN
4,NaN,17.5,NaN,NaN,NaN,67.666667,89.333333,NaN,NaN,NaN,...,NaN,NaN,55.0,100.0,NaN,NaN,NaN,67.0,NaN,NaN
6,NaN,100.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,65.0,...,NaN,38.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
